In [1]:
using SymbolicUtils
using Revise

In [10]:

# https://discourse.julialang.org/t/are-there-idioms-in-julia-for-fast-algebraic-data-types-adt/37244/20
module MyMod
abstract type Expression end
struct Add <: Expression
   x :: Expression
   y :: Expression
end

struct Lit <: Expression
    x :: Int64
end

evalu(a::Add) = evalu(a.x) + evalu(a.y)
evalu(l::Lit ) = l.x
end


MyMod.evalu( MyMod.Add(MyMod.Lit(3) , MyMod.Lit(7) ))

10

In [76]:
@syms x::Real y::Real c::Real z::Complex f(::Number)::Real diff2(::Real)::Real int2(::Real)::Real

(x, y, c, z, f(::Number)::Real, diff2(::Real)::Real, int2(::Real)::Real)

In [5]:
 2x^2 - y + x^2

(3 * (x ^ 2)) + (-1 * y)

In [40]:
x isa Number

false

In [88]:
#dplus = @rule +(~~xs) => map(fun )
dx = @rule diff2(x) => 1
#dexp = @rule diff(~x ^ ~n) => ~n * diff(~x) * ~x ^ (~n - 1) 
dmult = @rule diff2( ~x * ~y) => diff2(~x) * ~y + ~x * diff2(~y) 
dplus = @rule diff2( ~x + ~y ) => diff2(~x) + diff2(~y)
rset = RuleSet([dx,dmult,dplus])
rset(rset( diff2(x + x * x ) ))



#fixpoint(rset, 2 * (x + 1))

1 + (2 * x)

In [104]:
ipow = @rule int2(x ^ ~n) => x ^ (~n + 1) / (~n + 1) 

iconst = @rule int2( ~n::(n -> n isa Number) ) => ~n * x + c
ix = @rule int2(x) => x^2 / 2
iplus = @rule int2(+(~~y)) => sum(map(y -> int2(y) ,~~y))
imulconst = @rule int2( ~n::(n -> n isa Number) * ~x ) => ~n * int2(~x)
imulconst2 = @rule int2( ~x * ~n::(n -> n isa Number) ) => ~n * int2(~x)
imul = @rule int2(~u * ~dv) => ~u * int2(~dv) - int2(diff2(~u) * int2(~dv))
ifund = @rule int2(diff2(~x)) => ~x + c
idiffint = @rule diff2(int2(~x)) => ~x
icos = @rule int2(cos(x)) => sin(x) + c
isin = @rule int2(sin(x)) => -cos(x) + c
diffc = @rule diff2(c) => 0

# subs(x, ~a) => a
# subs(~f(~y) , ~a ) => f(subs(~y , ~a)

rset = RuleSet([iconst,ipow, ix, dx, imulconst, dmult, dplus, iplus, ifund, idiffint, isin, icos])
rset(rset( int2(3 + x + 7 + x ^ 3) ))
rset( int2(diff2( 3  )))
rset(imul(int2(x * x  )))
rset(simplify(int2(x * x)))



0.3333333333333333 * (x ^ 3)

integratin and differentiation are adjoint in the sense of linear algebra. a quadratcic form, you can integrate by parts. or cnsidering the finite difference matrix and it's transpose.  Are they adjoint in the sense of catgeoyr theory? From a symbolic perspective, they are both binding forms  (? integration this seems more straightforward. For derivative this is not obvious. This is what sicm says though. If we consider the function to be lambda bound. Then differentiation creates a variable, which you apply your original function to, but then it also outputs  a function.  ). There feels like a simlairty to the case of forall, exists, and weakening.

In [6]:
f(sin(x)^2 + cos(x)^2) + z

f(1) + z

In [7]:
r = @rule sinh(im * ~x) => sin(~x)

sinh(im * ~x) => sin(~x)

In [8]:
r(sinh(im * y))

sin(y)

In [2]:
:x

:x

In [4]:
quote a 
    b
    c
    d
end

quote
    #= In[4]:1 =#
    a
    #= In[4]:2 =#
    b
    #= In[4]:3 =#
    c
    #= In[4]:4 =#
    d
end

In [9]:
typeof(:(1 + x))

Expr

In [10]:
typeof(:x)

Symbol

In [13]:
1 == :1

true

In [16]:
:([1  2 3 ])

:([1 2 3])

In [21]:
fieldnames(typeof(:[1 2 3]))

(:head, :args)

In [25]:
:[1 2 3].args

3-element Array{Any,1}:
 1
 2
 3

In [27]:
:(1 + 2).args

3-element Array{Any,1}:
  :+
 1  
 2  

In [28]:
eval(:(1 + 2))

3

In [119]:
macro myvar(v)
    show(v)
    show(:($v = 3).head)
    return (v = 3)
end

@myvar (macro with 1 method)

In [120]:
@myvar z
z


:z:(=)

UndefVarError: UndefVarError: z not defined

In [107]:
x

3

In [105]:
:($(y) = 3)

:(3 = 3)

In [106]:
eval( :(x = 3)  )

3

In [88]:
macro diff(expr)
    show(expr)
    if typeof(expr) == Symbol
        if expr == :x
            return :1
        end
    elseif expr.head == :call
        
        if expr.args[1] == :+
            show(expr.args[2])
            show(expr.args[3])
            return :( @diff( $(expr.args[2])) + @diff($(expr.args[3]) )) #  :( sum(   [ diff(e) for e in expr.args[2:end] ]   ))
        end
    else
        return :foo
    end
end
    

@diff (macro with 1 method)

In [143]:
macroexpand(Main, :(@diff(x + x)))

:(x + x):x:x:x:x

:(1 + 1)

In [58]:

:(x + x).head 

:call

In [134]:


dom(::PlusIntCat) = ()
cod(::PlusIntCat) = ()
id( ::PlusIntCat ) = PlusIntCat(0)

Base.:∘(a::PlusIntCat, b::PlusIntCat) = PlusIntCat(a.n + b.n)

In [199]:
dom(::PlusIntCat) = PlusIntObj()
cod(::PlusIntCat) = PlusIntObj()
id( ::PlusIntObj ) = PlusIntCat(0)

id (generic function with 2 methods)

In [201]:
id(PlusIntObj())

PlusIntCat(0)

In [140]:
using Match

┌ Info: Precompiling Match [7eb4fadd-790c-5f42-8a69-bfa0b872bfbf]
└ @ Base loading.jl:1273


In [147]:
macroexpand( Main, quote function fact(n)
    @match n begin
    0 => 1
    n => n * fact(n-1)
    end
            end end)

quote
    #= In[147]:1 =#
    function fact(n)
        #= In[147]:2 =#
        begin
            #= In[147]:3 =#
            if Match.ismatch(0, n)
                #= /Users/philip/.julia/packages/Match/qiTCM/src/matchmacro.jl:408 =#
                1
            else
                #= /Users/philip/.julia/packages/Match/qiTCM/src/matchmacro.jl:410 =#
                begin
                    #= In[147]:4 =#
                    let n = n
                        n * fact(n - 1)
                    end
                end
            end
        end
    end
end

In [146]:
fact(3)

6

In [149]:
methods(first)

# 25 methods for generic function "first":
[1] first(::Tuple{}) in Base at tuple.jl:77
[2] first(s::BitSet) in Base at bitset.jl:353
[3] first(r::Core.Compiler.StmtRange) in Base.IRShow at show.jl:1599
[4] first(cmd::Cmd) in Base at process.jl:639
[5] first(x::Intervals.Inclusivity) in Intervals at /Users/philip/.julia/packages/Intervals/wasuB/src/inclusivity.jl:37
[6] first(t::Tuple) in Base at tuple.jl:78
[7] first(p::Pair) in Base at pair.jl:66
[8] first(r::Base.OneTo{T}) where T in Base at range.jl:566
[9] first(S::Base.Slice) in Base at indices.jl:329
[10] first(S::Base.IdentityUnitRange) in Base at indices.jl:361
[11] first(r::OrdinalRange{T,S} where S) where T in Base at range.jl:565
[12] first(r::StepRangeLen) in Base at range.jl:567
[13] first(r::LinRange) in Base at range.jl:568
[14] first(x::Number) in Base at number.jl:86
[15] first(iter::LinearIndices{1,R} where R<:Tuple{AbstractUnitRange{Int64}}) in Base at indices.jl:463
[16] first(iter::LinearIndices) in Base at indices.jl:462
[17] first(iter::CartesianIndices) in Base.IteratorsMD at multidimensional.jl:378
[18] first(a::AbstractArray) in Base at abstractarray.jl:324
[19] first(r::Base.Iterators.Reverse) in Base.Iterators at iterators.jl:86
[20] first(c::AbstractChar) in Base at char.jl:200
[21] first(::Base.EachStringIndex) in Base at strings/basic.jl:537
[22] first(s::AbstractString, n::Integer) in Base at strings/basic.jl:606
[23] first(interval::Intervals.Interval) in Intervals at /Users/philip/.julia/packages/Intervals/wasuB/src/interval.jl:112
[24] first(interval::Intervals.AnchoredInterval{P,T} where T) where P in Intervals at /Users/philip/.julia/packages/Intervals/wasuB/src/anchoredinterval.jl:129
[25] first(itr) in Base at abstractarray.jl:342

In [151]:
first((1,2))

1

In [152]:
last([1,2,3])

3

In [159]:
using JuMP

In [162]:
using GLPK

┌ Info: Precompiling GLPK [60bf3e95-4087-53dc-ae20-288a0d20c6a6]
└ @ Base loading.jl:1273


In [160]:
model = Model(GLPK.Optimizer)


UndefVarError: UndefVarError: GLPK not defined

In [163]:
macroexpand(Main, :(@variable(model, 0 <= y <= 30)))

quote
    #= /Users/philip/.julia/packages/JuMP/MsUSY/src/macros.jl:258 =#
    JuMP._valid_model(model, :model)
    #= /Users/philip/.julia/packages/JuMP/MsUSY/src/macros.jl:259 =#
    begin
        #= /Users/philip/.julia/packages/JuMP/MsUSY/src/macros.jl:316 =#
        JuMP._error_if_cannot_register(model, :y)
        #= /Users/philip/.julia/packages/JuMP/MsUSY/src/macros.jl:320 =#
        var"#19###422" = begin
                #= /Users/philip/.julia/packages/JuMP/MsUSY/src/macros.jl:274 =#
                let
                    #= /Users/philip/.julia/packages/JuMP/MsUSY/src/macros.jl:280 =#
                    var"#19###422" = JuMP.add_variable(model, JuMP.build_variable(JuMP.var"#_error#67"{Tuple{Symbol,Expr}}((:model, :(0 <= y <= 30))), JuMP.VariableInfo(true, 0, true, 30, false, NaN, false, NaN, false, false)), "y")
                    #= /Users/philip/.julia/packages/JuMP/MsUSY/src/macros.jl:281 =#
                    var"#19###422"
                end
            end
       

In [164]:


[1 2 3]

1×3 Array{Int64,2}:
 1  2  3

In [165]:
[1, 2, 3]

3-element Array{Int64,1}:
 1
 2
 3

In [166]:
[ 1 2 3 ; 3 4 5]

2×3 Array{Int64,2}:
 1  2  3
 3  4  5

In [167]:
[ [1,2 ] ,[3,4]]

2-element Array{Array{Int64,1},1}:
 [1, 2]
 [3, 4]

In [168]:
sigz = [ 1 0 ; 
         0 -1]

2×2 Array{Int64,2}:
 1   0
 0  -1

In [178]:
sigx = [ 0 1 
         1 0 ]

2×2 Array{Int64,2}:
 0  1
 1  0

In [179]:
sigy = [0   1im
       -1im  0 ]

2×2 Array{Complex{Int64},2}:
 0+0im  0+1im
 0-1im  0+0im

In [182]:
[ sigx  ; sigy ;  sigz ]

6×2 Array{Complex{Int64},2}:
 0+0im   1+0im
 1+0im   0+0im
 0+0im   0+1im
 0-1im   0+0im
 1+0im   0+0im
 0+0im  -1+0im

In [183]:
using LinearAlgebra

In [185]:
U, s, Vt = svd([ sigx  ; sigy ;  sigz ])

SVD{Complex{Float64},Float64,Array{Complex{Float64},2}}
U factor:
6×2 Array{Complex{Float64},2}:
      0.0+0.0im          0.57735+0.0im        
 -0.57735+0.0im      1.66533e-16+0.0im        
      0.0+0.0im              0.0+0.57735im    
      0.0+0.57735im          0.0-5.55112e-17im
 -0.57735+0.0im      5.55112e-17+0.0im        
      0.0+0.0im         -0.57735+0.0im        
singular values:
2-element Array{Float64,1}:
 1.7320508075688772
 1.7320508075688772
Vt factor:
2×2 Array{Complex{Float64},2}:
 -1.0+0.0im  -0.0+0.0im
  0.0+0.0im   1.0+0.0im

In [188]:
 kron(sigx, sigx)

4×4 Array{Int64,2}:
 0  0  0  1
 0  0  1  0
 0  1  0  0
 1  0  0  0

In [189]:
⊗ = kron

kron (generic function with 30 methods)

In [192]:
svd([ sigx ⊗ sigx ; sigy ⊗ sigy ; sigz ⊗ sigz ] )

SVD{Complex{Float64},Float64,Array{Complex{Float64},2}}
U factor:
12×4 Array{Complex{Float64},2}:
      0.0+0.0im       0.0+0.0im           0.0+0.0im       0.57735+0.0im
      0.0+0.0im       0.0+0.0im       0.57735+0.0im           0.0+0.0im
      0.0+0.0im  -0.57735+0.0im   1.66533e-16+0.0im           0.0+0.0im
 -0.57735+0.0im       0.0+0.0im           0.0+0.0im   1.66533e-16+0.0im
      0.0+0.0im       0.0+0.0im           0.0+0.0im      -0.57735+0.0im
      0.0+0.0im       0.0+0.0im       0.57735+0.0im           0.0+0.0im
      0.0+0.0im  -0.57735+0.0im   5.55112e-17+0.0im           0.0+0.0im
  0.57735+0.0im       0.0+0.0im           0.0+0.0im  -5.55112e-17+0.0im
 -0.57735+0.0im       0.0+0.0im           0.0+0.0im   5.55112e-17+0.0im
      0.0+0.0im   0.57735+0.0im  -5.55112e-17+0.0im           0.0+0.0im
      0.0+0.0im       0.0+0.0im      -0.57735+0.0im           0.0+0.0im
      0.0+0.0im       0.0+0.0im           0.0+0.0im       0.57735+0.0im
singular values:
4-element Array{Float

In [194]:
methods(∘)

# 2 methods for generic function "∘":
[1] ∘(a::PlusIntCat, b::PlusIntCat) in Main at In[134]:5
[2] ∘(f, g) in Base at operators.jl:873

Dict{Int64,Int64} with 1 entry:
  3 => 4

In [11]:
using Catlab

┌ Info: Precompiling Catlab [134e5e36-593f-5add-ad60-77f754baafbe]
└ @ Base loading.jl:1260


In [20]:

using Catlab.Doctrines
import Catlab.Doctrines: Ob, Hom, dom, codom, compose, ⋅, id, otimes, ⊗, munit, braid,
  hom, ev, curry, dual, dunit, dcounit, dagger, mate

In [17]:
struct PlusIntCat 
   n :: Int64
end

struct PlusIntObj end

In [21]:
@instance Category(PlusIntObj, PlusIntCat) begin
  dom(_::PlusIntCat) = PlusIntObj()
  codom(_::PlusIntCat) = PlusIntObj()
  #codom(M::Matrix) = MatrixDomain(eltype(M), size(M,2))

  id(_::PlusIntObj) = PlusIntCat(0)
  compose(M::PlusIntCat, N::PlusIntCat) = PlusIntCat(N.n + M.n)
end

In [22]:
compose(id(PlusIntObj()),  id(PlusIntObj()))

PlusIntCat(0)

In [14]:
methods(Ob)

# 23 methods for generic function "Ob":
[1] Ob(mod::Module, args...) in Catlab.Doctrines at /home/philip/.julia/packages/Catlab/G6PM3/src/doctrines/Doctrines.jl:14
[2] Ob(x1::Type{Catlab.Doctrines.FreeCategory.Ob}, x2) in Catlab.Doctrines
[3] Ob(x1::Type{Catlab.Doctrines.FreeCategory2.Ob}, x2) in Catlab.Doctrines
[4] Ob(x1::Type{Catlab.Doctrines.FreeSymmetricMonoidalCategory.Ob}, x2) in Catlab.Doctrines
[5] Ob(x1::Type{Catlab.Doctrines.FreeCartesianCategory.Ob}, x2) in Catlab.Doctrines
[6] Ob(x1::Type{Catlab.Doctrines.FreeBiproductCategory.Ob}, x2) in Catlab.Doctrines
[7] Ob(x1::Type{Catlab.Doctrines.FreeClosedMonoidalCategory.Ob}, x2) in Catlab.Doctrines
[8] Ob(x1::Type{Catlab.Doctrines.FreeCartesianClosedCategory.Ob}, x2) in Catlab.Doctrines
[9] Ob(x1::Type{Catlab.Doctrines.FreeCompactClosedCategory.Ob}, x2) in Catlab.Doctrines
[10] Ob(x1::Type{Catlab.Doctrines.FreeDaggerCategory.Ob}, x2) in Catlab.Doctrines
[11] Ob(x1::Type{Catlab.Doctrines.FreeDaggerSymmetricMonoidalCategory.Ob}, x2) in Catlab.Doctrines
[12] Ob(x1::Type{Catlab.Doctrines.FreeDaggerCompactCategory.Ob}, x2) in Catlab.Doctrines
[13] Ob(x1::Type{Catlab.Doctrines.FreeTracedMonoidalCategory.Ob}, x2) in Catlab.Doctrines
[14] Ob(x1::Type{Catlab.Doctrines.FreeThinCategory.Ob}, x2) in Catlab.Doctrines
[15] Ob(x1::Type{Catlab.Doctrines.FreeThinSymmetricMonoidalCategory.Ob}, x2) in Catlab.Doctrines
[16] Ob(x1::Type{Catlab.Doctrines.FreeAdditiveSymmetricMonoidalCategory.Ob}, x2) in Catlab.Doctrines
[17] Ob(x1::Type{Catlab.Doctrines.FreeCocartesianCategory.Ob}, x2) in Catlab.Doctrines
[18] Ob(x1::Type{Catlab.Doctrines.FreeBicategoryRelations.Ob}, x2) in Catlab.Doctrines
[19] Ob(x1::Type{Catlab.Doctrines.FreeAbelianBicategoryRelations.Ob}, x2) in Catlab.Doctrines
[20] Ob(x1::Type{Catlab.LinearAlgebra.GraphicalLinearAlgebra.FreeLinearFunctions.Ob}, x2) in Catlab.LinearAlgebra.GraphicalLinearAlgebra
[21] Ob(x1::Type{Catlab.LinearAlgebra.GraphicalLinearAlgebra.FreeLinearRelations.Ob}, x2) in Catlab.LinearAlgebra.GraphicalLinearAlgebra
[22] Ob(x1::Type{Catlab.LinearAlgebra.StructuredGraphicalLinearAlgebra.FreeStructuredLinearFunctions.Ob}, x2) in Catlab.LinearAlgebra.StructuredGraphicalLinearAlgebra
[23] Ob(typ::Type, args...) in Catlab.Doctrines at /home/philip/.julia/packages/Catlab/G6PM3/src/doctrines/Doctrines.jl:17

In [15]:
methods(id)

# 53 methods for generic function "id":
[1] id(A::Catlab.WiringDiagrams.WiringLayers.NLayer) in Catlab.WiringDiagrams.WiringLayers at /home/philip/.julia/packages/Catlab/G6PM3/src/wiring_diagrams/Layers.jl:160
[2] id(V::Catlab.LinearAlgebra.GraphicalLinearAlgebra.LinearMapDom) in Catlab.LinearAlgebra.GraphicalLinearAlgebra at /home/philip/.julia/packages/Catlab/G6PM3/src/linear_algebra/GLA.jl:136
[3] id(V::Catlab.LinearAlgebra.GraphicalLinearAlgebra.LinearOpDom) in Catlab.LinearAlgebra.GraphicalLinearAlgebra at /home/philip/.julia/packages/Catlab/G6PM3/src/linear_algebra/GLA.jl:177
[4] id(A::Catlab.Doctrines.Category.Ob) in Catlab.Doctrines
[5] id(x1::Catlab.Doctrines.FreeCategory.Ob) in Catlab.Doctrines
[6] id(A::Catlab.Doctrines.Category2.Ob) in Catlab.Doctrines
[7] id(f::Catlab.Doctrines.Category2.Hom2) in Catlab.Doctrines
[8] id(x1::Catlab.Doctrines.FreeCategory2.Ob) in Catlab.Doctrines
[9] id(x1::Catlab.Doctrines.FreeCategory2.Hom2) in Catlab.Doctrines
[10] id(A::Catlab.Doctrines.MonoidalCategory.Ob) in Catlab.Doctrines
[11] id(A::Catlab.Doctrines.SymmetricMonoidalCategory.Ob) in Catlab.Doctrines
[12] id(x1::Catlab.Doctrines.FreeSymmetricMonoidalCategory.Ob) in Catlab.Doctrines
[13] id(A::Catlab.Doctrines.MonoidalCategoryWithDiagonals.Ob) in Catlab.Doctrines
[14] id(A::Catlab.Doctrines.CartesianCategory.Ob) in Catlab.Doctrines
[15] id(x1::Catlab.Doctrines.FreeCartesianCategory.Ob) in Catlab.Doctrines
[16] id(A::Catlab.Doctrines.MonoidalCategoryWithBidiagonals.Ob) in Catlab.Doctrines
[17] id(A::Catlab.Doctrines.BiproductCategory.Ob) in Catlab.Doctrines
[18] id(x1::Catlab.Doctrines.FreeBiproductCategory.Ob) in Catlab.Doctrines
[19] id(A::Catlab.Doctrines.ClosedMonoidalCategory.Ob) in Catlab.Doctrines
[20] id(x1::Catlab.Doctrines.FreeClosedMonoidalCategory.Ob) in Catlab.Doctrines
[21] id(A::Catlab.Doctrines.CartesianClosedCategory.Ob) in Catlab.Doctrines
[22] id(x1::Catlab.Doctrines.FreeCartesianClosedCategory.Ob) in Catlab.Doctrines
[23] id(A::Catlab.Doctrines.CompactClosedCategory.Ob) in Catlab.Doctrines
[24] id(x1::Catlab.Doctrines.FreeCompactClosedCategory.Ob) in Catlab.Doctrines
[25] id(A::Catlab.Doctrines.DaggerCategory.Ob) in Catlab.Doctrines
[26] id(x1::Catlab.Doctrines.FreeDaggerCategory.Ob) in Catlab.Doctrines
[27] id(A::Catlab.Doctrines.DaggerSymmetricMonoidalCategory.Ob) in Catlab.Doctrines
[28] id(x1::Catlab.Doctrines.FreeDaggerSymmetricMonoidalCategory.Ob) in Catlab.Doctrines
[29] id(A::Catlab.Doctrines.DaggerCompactCategory.Ob) in Catlab.Doctrines
[30] id(x1::Catlab.Doctrines.FreeDaggerCompactCategory.Ob) in Catlab.Doctrines
[31] id(A::Catlab.Doctrines.TracedMonoidalCategory.Ob) in Catlab.Doctrines
[32] id(x1::Catlab.Doctrines.FreeTracedMonoidalCategory.Ob) in Catlab.Doctrines
[33] id(A::Catlab.Doctrines.ThinCategory.Ob) in Catlab.Doctrines
[34] id(x1::Catlab.Doctrines.FreeThinCategory.Ob) in Catlab.Doctrines
[35] id(A::Catlab.Doctrines.ThinSymmetricMonoidalCategory.Ob) in Catlab.Doctrines
[36] id(x1::Catlab.Doctrines.FreeThinSymmetricMonoidalCategory.Ob) in Catlab.Doctrines
[37] id(A::Catlab.Doctrines.AdditiveMonoidalCategory.Ob) in Catlab.Doctrines
[38] id(A::Catlab.Doctrines.AdditiveSymmetricMonoidalCategory.Ob) in Catlab.Doctrines
[39] id(x1::Catlab.Doctrines.FreeAdditiveSymmetricMonoidalCategory.Ob) in Catlab.Doctrines
[40] id(A::Catlab.Doctrines.MonoidalCategoryWithCodiagonals.Ob) in Catlab.Doctrines
[41] id(A::Catlab.Doctrines.CocartesianCategory.Ob) in Catlab.Doctrines
[42] id(x1::Catlab.Doctrines.FreeCocartesianCategory.Ob) in Catlab.Doctrines
[43] id(A::Catlab.Doctrines.BicategoryRelations.Ob) in Catlab.Doctrines
[44] id(x1::Catlab.Doctrines.FreeBicategoryRelations.Ob) in Catlab.Doctrines
[45] id(A::Catlab.Doctrines.AbelianBicategoryRelations.Ob) in Catlab.Doctrines
[46] id(x1::Catlab.Doctrines.FreeAbelianBicategoryRelations.Ob) in Catlab.Doctrines
[47] id(A::Catlab.WiringDiagrams.AlgebraicWiringDiagrams.Ports) in Catlab.WiringDiagrams.AlgebraicWiringDiagrams at /home/philip/.julia/packages/Catlab/G6